In [43]:
!pip install strands-agents[mistral] python-dotenv

In [18]:
import json
import os
import sys
import dotenv
import boto3
import requests

import pprint

import yaml
from pathlib import Path
from typing import Dict, List, Optional, Tuple, TypeVar
from tqdm import tqdm

from collections import Counter

# Pydantic for structured data
from pydantic import BaseModel, Field

# Strands for AI agents
from strands.agent import Agent
from strands.models.mistral import MistralModel

# AWS authentication
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest

sys.path.append('..')

from src.utils import (
    save_json,
    read_json,
    load_file_content,
    get_job_paths,
    get_training_paths,
    sanity_check,
	chat_with_persona,
    track_api_call,  # Cost tracking from utils
    print_cost_summary,  # Cost summary from utils
    reset_cost_tracker  # Reset cost tracker from utils
)

from src.my_utils import (
    display_markdown_file,
    call_mistral,
    get_agent,
    batch_extract,
    compute_stat_for_multi_items
)

from src.models.interview_info import(
    InterviewInfo,
    InverviewQualityInfo
)
from src.models.generic_models import ListOfIds
from src.models.job_info import JobInfo
from src.models.persona_info import PersonaInfo, PersonaSkills, RecommendationAnalysis
from src.models.activity_domain_info import ActivityDomainInfo, ListOfActivityDomains
from src.models.skill_domain_info import SkillDomainInfo, ListSkillsDomains
from src.prompts.persona_extraction_prompt import (
    PERSONA_EXTRACTION_PROMPT,
    PERSONA_INITIAL_EXTRACTION_PROMPT,
    PERSONA_EXTEND_SKILL_DOMAIN_PROMPT,
    PERSONA_SKILL_DOMAINS_CLASSIFICATION_PROMPT,
    PERSONA_ACTIVITY_DOMAINS_CLASSIFICATION_PROMPT,
    PERSONA_JOBS_EXTRACTION_PROMPT,
    PERSONA_JOB_SKILLS_EXTRACTION_PROMPT,
    PERSONA_INTEREST_EXTRACTION_PROMPT
)

# Load API key from .env file
dotenv.load_dotenv("../env")

# Check if we're good to go
if not os.getenv("MISTRAL_API_KEY"):
    print("❌ No MISTRAL_API_KEY found!")
    print("Create an env file with your API key")
else:
    print("✅ API key found, we're ready to roll")

✅ API key found, we're ready to roll


In [19]:
DATA_JOBS_DIR = Path('../data_jobs')
DATA_TRAININGS_DIR = Path('../data_trainings')
DATA_INTERVIEWS_DIR = Path('../data_interviews')
DATA_ACTIVITIES_DOMAINS_DIR = Path('../data_activities_domains')
DATA_SKILLS_DOMAINS_DIR = Path('../data_skills_domains')
DATA_PERSONAS_INFO_DIR = Path('../data_personas_info')
DATA_MATCH_JOBS_TRAININGS_DIR = Path('../data_match_jobs_trainings')

In [20]:
with open("../src/config.yaml", "r") as f:
    config = yaml.safe_load(f)

job_data_version = config["job_data_version"]
print(f"job_data_version version: {job_data_version}")

training_data_version = config["training_data_version"]
print(f"training_data_version version: {training_data_version}")

interview_data_version = config["interview_data_version"]
print(f"interview_data_version version: {interview_data_version}")

activity_domains_version = config["activity_domains_version"]
print(f"activity_domains_version version: {activity_domains_version}")

personas_info_data_version = config["personas_info_data_version"]
print(f"personas_info_data_version version: {personas_info_data_version}")

skill_domains_version = config["skill_domains_version"]
print(f"skill_domains_version version: {skill_domains_version}")

match_jobs_trainings_data_version = f"{job_data_version}_{training_data_version}"

job_data_version version: v4
training_data_version version: v7
interview_data_version version: v8
activity_domains_version version: v4
personas_info_data_version version: v14
skill_domains_version version: v3


In [21]:
# Load jobs domains map data
filename = f"map_clusters_jobs_{job_data_version}.json"
save_path = DATA_JOBS_DIR / filename
jobs_map = read_json(save_path)

In [22]:
# Load skills domains map data
filename = f"final_map_clusters_trainings_{training_data_version}.json"
save_path = DATA_TRAININGS_DIR / filename
trainings_map = read_json(save_path)

print(f"✅ Loaded {len(trainings_map)} skills domains")
print("\n" + "="*50)

✅ Loaded 12 skills domains



In [23]:
# Load Personas data
filename = f"job_filtered_personas_info_{personas_info_data_version}.json"
personas_save_path = DATA_PERSONAS_INFO_DIR / filename

initial_personas_data = read_json(personas_save_path)

# Convert to PersonaInfo objects
personas = {
    pid: PersonaInfo.model_validate_json(data)
    for pid, data in initial_personas_data.items()
}

print(f"✅ Loaded {len(personas)} personas")
print("\n" + "="*50)

✅ Loaded 100 personas



In [24]:
# Load jobs data
filename = f"final_jobs_{job_data_version}.json"
jobs_save_path = DATA_JOBS_DIR / filename
jobs_data = read_json(jobs_save_path)

# Convert to JobInfo objects
jobs_info = {
    job_id: JobInfo.model_validate_json(data)
    for job_id, data in jobs_data.items()
}

print(f"✅ Loaded {len(jobs_info)} jobs")
print("\n" + "="*50)

✅ Loaded 200 jobs



In [25]:
filename = f"match_jobs_trainings_{match_jobs_trainings_data_version}.json"
save_path = DATA_MATCH_JOBS_TRAININGS_DIR / filename
job_training_map = read_json(save_path)

In [26]:
filename = f"jobs_interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename
interviews = read_json(interviews_save_path)

# Convert to InterviewInfo objects
interviews_info = {}
for id in interviews:
    #print(id)
    interviews_info[id] = InterviewInfo(**interviews[id])

print(f"✅ Loaded {len(interviews_info)} interviews")
print("\n" + "="*50)


✅ Loaded 52 interviews



# Persona info parsing loop

In [27]:
def extract_jobs_info(
    conversation: List[str],
    jobs_str: str,
    model: str = "mistral-small-latest",
    print_prompt=False
) -> ListOfIds:
    """Extract persona info from conversation using Persona Extraction Agent"""
   
    text = '\n'.join(conversation)
    #print(text)

    prompt = PERSONA_JOBS_EXTRACTION_PROMPT.format(
        conversation=text,
        jobs_list=jobs_str
    )

    if print_prompt is True:
        print(prompt)
    
    extraction_agent = get_agent(model_id=model, temperature=0.0)
    result = extraction_agent.structured_output(output_model=ListOfIds, prompt=prompt)

    # Track cost
    if hasattr(extraction_agent, 'last_response'):
        track_api_call(extraction_agent.last_response, model)

    return result

In [29]:
def extract_skills_info(
    conversation: List[str],
    job_skills_str: str,
    model: str = "mistral-small-latest",
    print_prompt=False
) -> PersonaSkills:
    """Extract persona info from conversation using Persona Extraction Agent"""
   
    text = '\n'.join(conversation)
    #print(text)

    prompt = PERSONA_JOB_SKILLS_EXTRACTION_PROMPT.format(
        conversation=text,
        skills_list=job_skills_str
    )

    if print_prompt is True:
        print(prompt)

    extraction_agent = get_agent(model_id=model, temperature=0.0)
    result = extraction_agent.structured_output(output_model=PersonaSkills, prompt=prompt)

    if print_prompt is True:
        print(result)
        
    # Track cost
    if hasattr(extraction_agent, 'last_response'):
        track_api_call(extraction_agent.last_response, model)

    return result

In [31]:
def reset_personas_skills(personas_info_data_version):
    filename = f"personas_info_{personas_info_data_version}.json"
    personas_save_path = DATA_PERSONAS_INFO_DIR / filename
    personas_data = read_json(personas_save_path)

    for persona_id in personas_data:
        persona_data = json.loads(personas_data[persona_id])
        persona_data['skills'] = {}
        personas_data[persona_id] = json.dumps(persona_data, ensure_ascii=False)

    save_json(personas_save_path, personas_data)

In [33]:
def compute_skills_quality(persona_data):
    skills = persona_data['skills']

    if len(skills) == 0:
        issue_with_skills = True
    else:
        issue_with_skills = False

        for domain_skill in skills:
            level = skills[domain_skill]
            parts = domain_skill.split(" / ")
            if len(parts) < 2:
                issue_with_skills = True
                print("ERROR with skill name format")
                break

            domain = parts[0]
            skill = parts[1]
            if domain not in trainings_map:
                issue_with_skills = True
                print(f"ERROR with domain name : {domain}")
                break

            if skill not in trainings_map[domain]:
                issue_with_skills = True
                print(f"ERROR with skill name : {skill}")
                break

            if level not in ['None', 'Basic', 'Intermediate', 'Advanced']:
                issue_with_skills = True
                print(f"ERROR with level name : {level}")
                break

    return issue_with_skills

In [ ]:
MAX_LOOPS = 1
cache_period = 5

# Prepare personas info
filename = f"final_with_jobs_trainings_personas_info_{personas_info_data_version}.json"
personas_save_path = DATA_PERSONAS_INFO_DIR / filename
if not personas_save_path.exists():
    save_json(personas_save_path, {})
personas_data = read_json(personas_save_path)

new_items_processed = 0

print("START CLASSIFICATION LOOP")
for i in range(MAX_LOOPS):
    print("ITERATE CLASSIFICATION LOOP")

    filt_personas_ids = []
    for persona_id in initial_personas_data:
        if persona_id not in personas_data:
            persona_data = json.loads(initial_personas_data[persona_id])
            filt_personas_ids.append(persona_id)
        else:
            persona_data = json.loads(initial_personas_data[persona_id])
            if persona_data['recommendation_type'] == 'jobs_trainings':
                persona_data = json.loads(personas_data[persona_id])
                issue_with_skills = compute_skills_quality(persona_data)
                if issue_with_skills is True:
                    filt_personas_ids.append(persona_id)

        #print(persona_id)
        #print(persona_data)
        personas_data[persona_id] = json.dumps(persona_data, ensure_ascii=False)

    print(f'Personas to process: {filt_personas_ids}')

    if len(filt_personas_ids) == 0:
        break

    #print(filt_personas_ids)
    #print(personas_data['persona_004'])
    
    # Reset cost tracker if starting fresh
    # if len(persona_infos) == 0:
    #     reset_cost_tracker()
    #     print("💰 Starting fresh - cost tracker reset")

    # Track how many new personas we process
    new_personas_processed = 0

    for persona_id in tqdm(filt_personas_ids):
        if persona_id not in interviews_info:
            continue

        #print(persona_id)
        
        new_personas_processed += 1
        conversation = interviews_info[persona_id].interview
        # print(conversation)

        print_prompt = False
        #if new_personas_processed == 1:
        #    print_prompt = True

        persona_data = json.loads(personas_data[persona_id])

        jobs_str = ""
        for job_id in persona_data['proposed_job_ids']:
            jobs_str += f"- '{job_id}'" + "\n"
    
        result = extract_jobs_info(
            interviews_info[persona_id].interview,
            jobs_str,
            model="mistral-medium-latest",
            print_prompt=print_prompt
        )

        #print(result)
        #persona_data['selected_job_ids'] = result.list_of_ids

        if 'NOT_INTERESTED' in result.list_of_ids:
            persona_data['recommendation_type'] = 'trainings_only'
        else:
            job_skills_str = ""
            for job_id in persona_data['proposed_job_ids']:
                job_trainings = job_training_map[job_id]
                for training in job_trainings:
                    job_skills_str += f"- {training} : {job_trainings[training]}" + "\n"
    
            # print(job_skills_str)
            # print(interviews_info[persona_id].interview)
    
            result = extract_skills_info(
                interviews_info[persona_id].interview,
                job_skills_str,
                model="mistral-medium-latest",
                print_prompt=print_prompt
            )
    
            #print(result)
            #persona_data = json.loads(personas_data[persona_id])
            persona_data['skills'] = result.skills
            
        personas_data[persona_id] = json.dumps(persona_data, ensure_ascii=False)
        # print(persona_data)

        # Save every 5 personas
        if new_personas_processed % 5 == 0:
            save_json(personas_save_path, personas_data)

        # Show cost update every 20 personas
        if new_personas_processed > 0 and new_personas_processed % 20 == 0:
            print(f"\n💰 Cost update after {new_personas_processed} new personas:")
            print_cost_summary()
            print()

        # if new_personas_processed > 4:break

save_json(personas_save_path, personas_data)

# Convert to PersonaInfo objects
# personas = {
#     pid: PersonaInfo.model_validate_json(data)
#     for pid, data in persona_infos.items()
# }

# print(f"\n✅ Interviewed {len(personas)} personas total ({new_personas_processed} new)")


---

# For Debug Only

---

In [28]:
if False:
    persona_id = 'persona_033'
    persona_info = personas[persona_id]
    print(persona_info.proposed_job_ids)

    if len(persona_info.proposed_job_ids) > 0:
        jobs_str = ""
        for job_id in persona_info.proposed_job_ids:
            jobs_str += f"- '{job_id}'" + "\n"

        # print(interviews_info[persona_id].interview)

        result = extract_jobs_info(
            interviews_info[persona_id].interview,
            jobs_str,
            model="mistral-medium-latest",
            print_prompt=True
        )

        print(result)

In [30]:
if False:
    persona_id = 'persona_010'
    persona_info = personas[persona_id]
    print(persona_info.proposed_job_ids)

    if len(persona_info.proposed_job_ids) > 0:
        job_skills_str = ""
        for job_id in persona_info.proposed_job_ids:
            job_trainings = job_training_map[job_id]
            for training in job_trainings:
                job_skills_str += f"- {training} : {job_trainings[training]}" + "\n"

        print(job_skills_str)
        # print(interviews_info[persona_id].interview)

        result = extract_skills_info(
            interviews_info[persona_id].interview,
            job_skills_str,
            model="mistral-medium-latest",
            print_prompt=False
        )

        print(result)

In [32]:
if False:
    reset_personas_skills(personas_info_data_version)

# Redo persona

In [35]:
persona_id = 'persona_041'

filename = f"final_with_jobs_trainings_personas_info_{personas_info_data_version}.json"
personas_save_path = DATA_PERSONAS_INFO_DIR / filename
personas_data = read_json(personas_save_path)

# print(initial_personas_data[persona_id])

persona_data = json.loads(initial_personas_data[persona_id])
if persona_data['recommendation_type'] != "jobs_trainings":
    personas_data[persona_id] = initial_personas_data[persona_id]
    print(initial_personas_data[persona_id])
    save_json(personas_save_path, personas_data)  
elif persona_id not in interviews_info:
    print('ISSUE')
else:
    conversation = interviews_info[persona_id].interview
    # print(conversation)

    persona_data = json.loads(initial_personas_data[persona_id])

    jobs_str = ""
    for job_id in persona_data['proposed_job_ids']:
        jobs_str += f"- '{job_id}'" + "\n"

    result = extract_jobs_info(
        interviews_info[persona_id].interview,
        jobs_str,
        model="mistral-medium-latest",
        print_prompt=False
    )

    print(result)

    if 'NOT_INTERESTED' in result.list_of_ids:
        persona_data['recommendation_type'] = 'trainings_only'
    else:
        print('HERE')
        job_skills_str = ""
        for job_id in persona_data['proposed_job_ids']:
            job_trainings = job_training_map[job_id]
            for training in job_trainings:
                job_skills_str += f"- {training} : {job_trainings[training]}" + "\n"
    
        result = extract_skills_info(
            interviews_info[persona_id].interview,
            job_skills_str,
            model="mistral-large-latest",
            print_prompt=True
        )
    
        print(result)
        
        persona_data['skills'] = result.skills

    personas_data[persona_id] = json.dumps(persona_data, ensure_ascii=False)
    print(persona_data)

    save_json(personas_save_path, personas_data)


list_of_ids=['NOT_INTERESTED'] rationale='The user explicitly mentioned that they are not looking for a job right away. Instead, they are interested in finding training programs to improve their skills and prepare for a future role in tourism.'
{'name': '', 'age': 19, 'location': 'Belo Horizonte', 'recommendation_type': 'trainings_only', 'open_to_relocate_for_work': False, 'work_type_preference': 'onsite', 'target_domains': ['Tourism And Hospitality Guest Services'], 'education_level': 'Técnico', 'years_of_experience': 0, 'skills_domains': [], 'skills': {}, 'languages': ['Portuguese', 'English'], 'goals': 'work with people and languages, practice English more, possibly with travelers, and contribute to hospitality.', 'hard_filtered_jobs_ids': [], 'proposed_job_ids': []}


# Persona info quality

In [ ]:
filename = f"jobs_interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename
interviews = read_json(interviews_save_path)

# Convert to InterviewInfo objects
interviews_info = {}
for id in interviews:
    interviews_info[id] = InterviewInfo(**interviews[id])

print(f"✅ Loaded {len(interviews_info)} interviews")
print("\n" + "="*50)


In [ ]:
filename = f"personas_info_{personas_info_data_version}.json"
personas_save_path = DATA_PERSONAS_INFO_DIR / filename
personas_data = read_json(personas_save_path)

In [19]:
for persona_id in interviews_info:
    persona_data = json.loads(personas_data[persona_id])
    compute_skills_quality(persona_data)


ERROR with level name : 
ERROR with level name : 
ERROR with level name : 
ERROR with skill name format
ERROR with skill name format
ERROR with skill name format
ERROR with skill name format
ERROR with level name : Basic to Intermediate


# Persona info statistics

In [45]:
filename = f"personas_info_{personas_info_data_version}.json"
personas_save_path = DATA_PERSONAS_INFO_DIR / filename
persona_infos = read_json(personas_save_path)

In [46]:
# 1. Extract ages from the JSON data
ages = []
for persona in persona_infos.values():
    data = json.loads(persona)
    if data['age'] > 0:  # Filter out invalid age (0)
        ages.append(data['age'])
age_counts = Counter(ages)
# Print in ascending order of age
for age in sorted(age_counts.keys()):
    print(f"Age {age}: {age_counts[age]} occurrences")

# Extract infos from the dictionary
recommendation_types = []
open_to_relocate_for_work = []
work_type_preferences = []
locations = []
years_of_experiences = []
target_domains = []
skills_domains = []
for persona in persona_infos.values():
    data = json.loads(persona)
    recommendation_types.append(data['recommendation_type'])
    open_to_relocate_for_work.append(data['open_to_relocate_for_work'])
    work_type_preferences.append(data['work_type_preference'])
    locations.append(data['location'])
    years_of_experiences.append(data['years_of_experience'])
    for domain in data['target_domains']:
        target_domains.append(domain)
    for domain in data['skills_domains']:
        skills_domains.append(domain)

type_counts = Counter(recommendation_types)
print("\nRecommendation Type Counts:")
for type_name, count in type_counts.most_common():
    print(f"{type_name}: {count}")
print(f"\nTotal count: {sum(type_counts.values())}")

type_counts = Counter(open_to_relocate_for_work)
print("\nopen_to_relocate_for_work Counts:")
for type_name, count in type_counts.most_common():
    print(f"{type_name}: {count}")
print(f"\nTotal count: {sum(type_counts.values())}")

type_counts = Counter(work_type_preferences)
print("\nWork type preferences Counts:")
for type_name, count in type_counts.most_common():
    print(f"{type_name}: {count}")
print(f"\nTotal count: {sum(type_counts.values())}")

type_counts = Counter(years_of_experiences)
print("\nyears_of_experience Counts:")
for type_name, count in type_counts.most_common():
    print(f"{type_name}: {count}")
print(f"\nTotal count: {sum(type_counts.values())}")

type_counts = Counter(locations)
print("\nLocation Counts:")
for type_name, count in type_counts.most_common():
    print(f"{type_name}: {count}")
print(f"\nTotal count: {sum(type_counts.values())}")

type_counts = Counter(target_domains)
print("\ntarget_domains Counts:")
for type_name, count in type_counts.most_common():
    print(f"{type_name}: {count}")
print(f"\nTotal count: {sum(type_counts.values())}")

type_counts = Counter(skills_domains)
print("\nskills_domains Counts:")
for type_name, count in type_counts.most_common():
    print(f"{type_name}: {count}")
print(f"\nTotal count: {sum(type_counts.values())}")

"""
item_counts, value_counts = compute_stat_for_multi_items(persona_infos, 'skills', include_values=True)
print("\n" + "="*50)
print("\nSkills Values Counts:")
print(f"\nTotal occurrences: {sum(value_counts.values())}")
print(f"Total unique types: {len(value_counts)}")
for item, count in value_counts.most_common():
    print(f"{item}: {count}")
print("\nSkills Counts:")
print(f"\nTotal occurrences: {sum(item_counts.values())}")
print(f"Total unique types: {len(item_counts)}")
for item, count in item_counts.most_common():
    print(f"{item}: {count}")
"""

Age 13: 2 occurrences
Age 14: 1 occurrences
Age 15: 5 occurrences
Age 16: 11 occurrences
Age 17: 5 occurrences
Age 18: 12 occurrences
Age 19: 8 occurrences
Age 20: 2 occurrences
Age 21: 5 occurrences
Age 22: 6 occurrences
Age 23: 4 occurrences
Age 24: 2 occurrences
Age 25: 15 occurrences
Age 26: 5 occurrences
Age 27: 7 occurrences
Age 28: 4 occurrences
Age 29: 1 occurrences
Age 30: 1 occurrences

Recommendation Type Counts:
jobs_trainings: 51
trainings_only: 38
awareness: 11

Total count: 100

open_to_relocate_for_work Counts:
False: 89
True: 11

Total count: 100

Work type preferences Counts:
onsite: 74
: 12
hybrid: 8
any: 4
remote: 2

Total count: 100

years_of_experience Counts:
0: 40
1: 25
2: 24
3: 11

Total count: 100

Location Counts:
São Paulo: 15
Recife: 13
Brasília: 13
Rio de Janeiro: 12
Belo Horizonte: 11
Curitiba: 10
Salvador: 9
Porto Alegre: 8
Fortaleza: 5
Brazil: 3
: 1

Total count: 100

target_domains Counts:
Electrical And Electronics Systems: 6
Cultural Heritage Informa

'\nitem_counts, value_counts = compute_stat_for_multi_items(persona_infos, \'skills\', include_values=True)\nprint("\n" + "="*50)\nprint("\nSkills Values Counts:")\nprint(f"\nTotal occurrences: {sum(value_counts.values())}")\nprint(f"Total unique types: {len(value_counts)}")\nfor item, count in value_counts.most_common():\n    print(f"{item}: {count}")\nprint("\nSkills Counts:")\nprint(f"\nTotal occurrences: {sum(item_counts.values())}")\nprint(f"Total unique types: {len(item_counts)}")\nfor item, count in item_counts.most_common():\n    print(f"{item}: {count}")\n'

In [19]:
# Correct Persona Info Extraction
print("💼 Testing Persona information Extraction Agent...")
print("Reading a sample job file...\n")

filename = f"interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename
interviews = read_json(interviews_save_path)

filename = f"personas_info_{personas_info_data_version}.json"
personas_save_path = DATA_PERSONAS_INFO_DIR / filename
personas_data = read_json(personas_save_path)

# Get first job file
persona_id = "persona_015"

if persona_id in interviews:
    conversation = interviews[persona_id]

    # Extract
    persona_info = extract_persona_info(activities_domains, skills_domains, conversation)
    personas_data[persona_id] = persona_info.model_dump_json()
    save_json(personas_save_path, personas_data)

    print(persona_info)
    #persona_infos[persona_id] = persona_info.model_dump_json()
    
    # persona_info = extract_persona_info(activities_domains, skills_domains, conversation)
   

💼 Testing Persona information Extraction Agent...
Reading a sample job file...

name='Camila' age=22 location='Fortaleza' recommendation_type='trainings_only' open_to_relocate_for_work=False work_type_preference='onsite' target_domains=['UNKNOWN'] education_level='Técnico' years_of_experience=0 skills_domains=['UNKNOWN'] skills={} languages={'Portuguese', 'English'} goals='Learn all basics of live event production: lights, sound, stage setup.'
